# Actividad de repaso

In [79]:
import numpy as np
import pandas as pd
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
import matplotlib.pyplot as plt
import statsmodels.api as sm
from scipy import stats 
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error
from sklearn.linear_model import Ridge, ElasticNet, Lasso, LinearRegression, LogisticRegression
from sklearn.model_selection import LeaveOneOut, train_test_split, KFold, cross_val_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn import datasets
from sklearn.compose import ColumnTransformer
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer

In [2]:
data= pd.read_csv("/Users/paofigueroa/Documents/sem 5/Lab de aprendizaje estadístico/brain_tumor_dataset.csv")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Patient_ID           20000 non-null  int64  
 1   Age                  20000 non-null  int64  
 2   Gender               20000 non-null  object 
 3   Tumor_Type           20000 non-null  object 
 4   Tumor_Size           20000 non-null  float64
 5   Location             20000 non-null  object 
 6   Histology            20000 non-null  object 
 7   Stage                20000 non-null  object 
 8   Symptom_1            20000 non-null  object 
 9   Symptom_2            20000 non-null  object 
 10  Symptom_3            20000 non-null  object 
 11  Radiation_Treatment  20000 non-null  object 
 12  Surgery_Performed    20000 non-null  object 
 13  Chemotherapy         20000 non-null  object 
 14  Survival_Rate        20000 non-null  float64
 15  Tumor_Growth_Rate    20000 non-null 

In [3]:
data.isna()

,Patient_ID,Age,Gender,Tumor_Type,Tumor_Size,Location,Histology,Stage,Symptom_1,Symptom_2,Symptom_3,Radiation_Treatment,Surgery_Performed,Chemotherapy,Survival_Rate,Tumor_Growth_Rate,Family_History,MRI_Result,Follow_Up_Required
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
19996,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
19997,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
19998,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [4]:
data.drop_duplicates()

,Patient_ID,Age,Gender,Tumor_Type,Tumor_Size,Location,Histology,Stage,Symptom_1,Symptom_2,Symptom_3,Radiation_Treatment,Surgery_Performed,Chemotherapy,Survival_Rate,Tumor_Growth_Rate,Family_History,MRI_Result,Follow_Up_Required
0,1,73,Male,Malignant,5.375612,Temporal,Astrocytoma,III,Vision Issues,Seizures,Seizures,No,No,No,51.312579,0.111876,No,Positive,Yes
1,2,26,Male,Benign,4.847098,Parietal,Glioblastoma,II,Headache,Headache,Nausea,Yes,Yes,Yes,46.373273,2.165736,Yes,Positive,Yes
2,3,31,Male,Benign,5.588391,Parietal,Meningioma,I,Vision Issues,Headache,Seizures,No,No,No,47.072221,1.884228,No,Negative,No
3,4,29,Male,Malignant,1.436600,Temporal,Medulloblastoma,IV,Vision Issues,Seizures,Headache,Yes,No,Yes,51.853634,1.283342,Yes,Negative,No
4,5,54,Female,Benign,2.417506,Parietal,Glioblastoma,I,Headache,Headache,Seizures,No,No,Yes,54.708987,2.069477,No,Positive,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,19996,21,Male,Malignant,9.612013,Parietal,Medulloblastoma,III,Seizures,Nausea,Vision Issues,No,No,Yes,58.229662,0.353806,No,Negative,Yes
19996,19997,32,Female,Benign,1.543560,Temporal,Meningioma,III,Headache,Headache,Vision Issues,Yes,Yes,No,77.706856,2.341074,No,Positive,No
19997,19998,57,Female,Benign,3.618634,Temporal,Medulloblastoma,I,Seizures,Vision Issues,Nausea,No,No,Yes,89.543803,2.332881,No,Positive,Yes
19998,19999,68,Male,Malignant,8.519086,Parietal,Glioblastoma,III,Seizures,Headache,Vision Issues,Yes,Yes,Yes,83.306781,2.387202,No,Positive,No


In [5]:
data.columns

Index(['Patient_ID', 'Age', 'Gender', 'Tumor_Type', 'Tumor_Size', 'Location',
       'Histology', 'Stage', 'Symptom_1', 'Symptom_2', 'Symptom_3',
       'Radiation_Treatment', 'Surgery_Performed', 'Chemotherapy',
       'Survival_Rate', 'Tumor_Growth_Rate', 'Family_History', 'MRI_Result',
       'Follow_Up_Required'],
      dtype='object')

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Patient_ID           20000 non-null  int64  
 1   Age                  20000 non-null  int64  
 2   Gender               20000 non-null  object 
 3   Tumor_Type           20000 non-null  object 
 4   Tumor_Size           20000 non-null  float64
 5   Location             20000 non-null  object 
 6   Histology            20000 non-null  object 
 7   Stage                20000 non-null  object 
 8   Symptom_1            20000 non-null  object 
 9   Symptom_2            20000 non-null  object 
 10  Symptom_3            20000 non-null  object 
 11  Radiation_Treatment  20000 non-null  object 
 12  Surgery_Performed    20000 non-null  object 
 13  Chemotherapy         20000 non-null  object 
 14  Survival_Rate        20000 non-null  float64
 15  Tumor_Growth_Rate    20000 non-null 

Vamos a intentar hacer una regresión logística para sacar los p-values y determinar las variables más significativas del modelo

In [7]:
num_columns= ["Patient_ID", "Age", "Tumor_Size","Survival_Rate","Tumor_Growth_Rate"]
cat_features= ["Gender", "Location","Histology", "Stage", "Symptom_1", "Symptom_2", "Symptom_3", "Radiation_Treatment", "Surgery_Performed", "Chemotherapy","Family_History", "MRI_Result", "Follow_Up_Required"]
Target=["Tumor_Type"]
print("Variables numéricas: ", num_columns)
print("Variables categóricas: ", cat_features)
print("Target: ", Target)

Variables numéricas:  ['Patient_ID', 'Age', 'Tumor_Size', 'Survival_Rate', 'Tumor_Growth_Rate']
Variables categóricas:  ['Gender', 'Location', 'Histology', 'Stage', 'Symptom_1', 'Symptom_2', 'Symptom_3', 'Radiation_Treatment', 'Surgery_Performed', 'Chemotherapy', 'Family_History', 'MRI_Result', 'Follow_Up_Required']
Target:  ['Tumor_Type']


In [8]:
data["Tumor_Type"]

0        Malignant
1           Benign
2           Benign
3        Malignant
4           Benign
           ...    
19995    Malignant
19996       Benign
19997       Benign
19998    Malignant
19999       Benign
Name: Tumor_Type, Length: 20000, dtype: object

In [9]:
data=data.sample(frac=0.1, random_state=42)

In [10]:
X= data[num_columns + cat_features]
X1=pd.get_dummies(X, drop_first=True)
y_transformada= data['Tumor_Type']=='Benign'
y_transformada
y= y_transformada.astype(int)

In [11]:
print(X1.info())

<class 'pandas.core.frame.DataFrame'>
Index: 2000 entries, 10650 to 17323
Data columns (total 30 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Patient_ID                 2000 non-null   int64  
 1   Age                        2000 non-null   int64  
 2   Tumor_Size                 2000 non-null   float64
 3   Survival_Rate              2000 non-null   float64
 4   Tumor_Growth_Rate          2000 non-null   float64
 5   Gender_Male                2000 non-null   bool   
 6   Location_Occipital         2000 non-null   bool   
 7   Location_Parietal          2000 non-null   bool   
 8   Location_Temporal          2000 non-null   bool   
 9   Histology_Glioblastoma     2000 non-null   bool   
 10  Histology_Medulloblastoma  2000 non-null   bool   
 11  Histology_Meningioma       2000 non-null   bool   
 12  Stage_II                   2000 non-null   bool   
 13  Stage_III                  2000 non-null   bool 

patient_Id es un identificador, no una variable o característica del modelo, por lo que la podemos quitar

In [12]:
X1_calc = X1.drop('Patient_ID', axis=1)
X1_calc = X1_calc.astype(float)

In [13]:
X1_scaled = X1_calc.copy()
num_features_to_scale = ['Age', 'Tumor_Size', 'Survival_Rate', 'Tumor_Growth_Rate']

scaler = StandardScaler()
X1_scaled[num_features_to_scale] = scaler.fit_transform(X1_scaled[num_features_to_scale])

## Primer intento de regresión

In [14]:
# Regresión logistica
model = LogisticRegression()
model.fit(X1_calc, y)
print("Intercepto: ",model.intercept_)
print("Coeficiente: ", model.coef_)

Intercepto:  [0.10492193]
Coeficiente:  [[ 9.47868949e-04  1.68809500e-02 -5.54809861e-04  6.44134908e-03
   5.43870795e-03  1.87718465e-01  3.16530552e-02 -1.45238177e-01
   1.04098368e-02 -1.10673270e-01 -1.35483781e-01  3.75505297e-02
   1.02495753e-01 -2.83214899e-02  1.86416980e-02  3.51814990e-03
  -1.68530059e-01 -1.41269166e-01 -5.05854824e-02 -1.20301153e-01
   4.47057889e-02 -8.51200999e-02 -1.61783733e-01 -7.09353336e-02
   9.30331872e-02 -7.29858960e-02 -1.43419580e-04 -4.28602532e-02
  -1.12072171e-02]]


/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [15]:
import statsmodels.api as sm

X_con_constante = sm.add_constant(X1_calc)
logit_model = sm.Logit(y, X_con_constante)
result = logit_model.fit()
print(result.summary())

Optimization terminated successfully.
         Current function value: 0.687719
         Iterations 4
                           Logit Regression Results                           
Dep. Variable:             Tumor_Type   No. Observations:                 2000
Model:                          Logit   Df Residuals:                     1970
Method:                           MLE   Df Model:                           29
Date:                Mon, 27 Oct 2025   Pseudo R-squ.:                0.007805
Time:                        19:30:54   Log-Likelihood:                -1375.4
converged:                       True   LL-Null:                       -1386.3
Covariance Type:            nonrobust   LLR p-value:                    0.8348
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                         0.2047      0.352      0.582      0.561      -0.

## Segundo intento de regresión

In [16]:
from scipy.stats import norm
import statsmodels.api as sm 


y_pred = model.predict_proba(X1_scaled)[:, 1]
p = y_pred
p_menos1 = p * (1 - p)
V = np.diagflat(p_menos1)


X_const_ = sm.add_constant(X1_scaled, prepend=True) 
X_const_np = X_const_.values 
V_np = V


cov_full = np.linalg.inv(X_const_np.T @ V_np @ X_const_np)


se_full = np.sqrt(np.diag(cov_full))
all_coefs = np.hstack([model.intercept_, model.coef_.flatten()])
z = all_coefs / se_full
p_value = 2 * (1 - norm.cdf(abs(z)))

p_values_df = pd.DataFrame({
    'Feature': X_const_.columns,
    'p_value': p_value
})

print(p_values_df.sort_values(by='p_value'))

                      Feature   p_value
6          Location_Occipital  0.149238
17    Symptom_1_Vision Issues  0.185572
23    Symptom_3_Vision Issues  0.202865
8           Location_Temporal  0.262607
18           Symptom_2_Nausea  0.268965
11       Histology_Meningioma  0.293920
25      Surgery_Performed_Yes  0.304568
20    Symptom_2_Vision Issues  0.355975
10  Histology_Medulloblastoma  0.390506
26           Chemotherapy_Yes  0.421442
13                  Stage_III  0.430973
24    Radiation_Treatment_Yes  0.433443
22         Symptom_3_Seizures  0.511938
28        MRI_Result_Positive  0.636932
0                       const  0.647743
19         Symptom_2_Seizures  0.690821
2                  Tumor_Size  0.709102
21           Symptom_3_Nausea  0.723319
12                   Stage_II  0.768697
7           Location_Parietal  0.803926
14                   Stage_IV  0.827793
15           Symptom_1_Nausea  0.884313
4           Tumor_Growth_Rate  0.887013
29     Follow_Up_Required_Yes  0.901512


De acuerdo con ambos intentos, no hay ninguna variable que sea completamente significativa para el modelo en este "arrange", por lo que mejor lo haré por prueba y error. Podríamos intentar con interacciones de variables ya que quizás la variable 1 por sí sola no explique nada, pero con un polinomio, función u otra variable, podría cobrar sentido

In [80]:
numerical_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

In [82]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, num_columns),
        ('cat', categorical_transformer, cat_features)
    ])

In [83]:
svc=SVC(kernel='linear', probability=True, C=1)

In [85]:
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('svc', SVC(kernel='linear', probability=True)) # Puedes quitar C=1.0, es el default
])

In [86]:
pipeline.fit(X,y)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['Patient_ID', 'Age',
                                                   'Tumor_Size',
                                                   'Survival_Rate',
                                                   'Tumor_Growth_Rate']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Gender', 'Location',
                                                   'Histology', 'Stage',
                                                   'Symptom_1', 'Symptom_2',
                                                   'Symptom_3',
                                                   'Radiation_Treatment',
                                                   'Surgery_Performed',
                                                   'Chemotherapy',
                                                   'Family_History',
                                                   'MRI_Result',
                                                   'Follow_Up_Required'])])),
                ('svc', SVC(kernel='linear', probability=True))])

In [87]:
pipeline.score(X,y)

0.537

In [88]:
kfolds=10
scores=cross_val_score(pipeline,X,y,
                       scoring='roc_auc_ovr',
                       cv=kfolds)

In [89]:
mean_score=np.mean(scores)
mean_score

0.5063508195819583

In [90]:
std_score=np.std(scores)
std_score

0.03235602841643138

## Optimización Bayesiana

In [91]:
search_spaces = {
    'svc__C': Real(1e-4, 1e+4, prior='log-uniform') 
}
cv_strategy = KFold(n_splits=10, shuffle=True, random_state=42)

opt = BayesSearchCV(
    estimator=pipeline,
    search_spaces=search_spaces,
    n_iter=10,  # 50 iteraciones de búsqueda
    scoring='roc_auc',
    cv=cv_strategy,
    n_jobs=-1,
    random_state=42,
    verbose=1 # Muestra el progreso
)
opt.fit(X, y)

Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits


BayesSearchCV(cv=KFold(n_splits=10, random_state=42, shuffle=True),
              estimator=Pipeline(steps=[('preprocessor',
                                         ColumnTransformer(transformers=[('num',
                                                                          StandardScaler(),
                                                                          ['Patient_ID',
                                                                           'Age',
                                                                           'Tumor_Size',
                                                                           'Survival_Rate',
                                                                           'Tumor_Growth_Rate']),
                                                                         ('cat',
                                                                          OneHotEncoder(handle_unknown='ignore'),
                                                                          ['Gender',
                                                                           'Location',
                                                                           'Histology',
                                                                           'Stage',
                                                                           'Symptom_1',
                                                                           'Symptom_2',
                                                                           'Symptom_3',
                                                                           'Radiation_Treatment',
                                                                           'Surgery_Performed',
                                                                           'Chemotherapy',
                                                                           'Family_History',
                                                                           'MRI_Result',
                                                                           'Follow_Up_Required'])])),
                                        ('svc',
                                         SVC(kernel='linear',
                                             probability=True))]),
              n_iter=10, n_jobs=-1, random_state=42, scoring='roc_auc',
              search_spaces={'svc__C': Real(low=0.0001, high=10000.0, prior='log-uniform', transform='normalize')},
              verbose=1)

In [92]:
print(f"El mejor score (ROC AUC) es: {opt.best_score_}")
print(f"Los mejores hiperparámetros son: {opt.best_params_}")

El mejor score (ROC AUC) es: 0.49471370138433296
Los mejores hiperparámetros son: OrderedDict({'svc__C': 4403.762515730915})


vamos a probar con ese C

In [93]:
svc=SVC(kernel='linear', probability=True, C=4403.762515730915)
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('svc', SVC(kernel='linear', probability=True))
])
pipeline.fit(X,y)
pipeline.score(X,y)
print(pipeline.score(X,y))

kfolds=10
scores=cross_val_score(pipeline,X,y,
                       scoring='roc_auc_ovr',
                       cv=kfolds)

0.537


In [73]:
mean_score=np.mean(scores)
mean_score

0.5008503260326033

In [74]:
std_score=np.std(scores)
std_score

0.03594544061910024

Hacemos la optimización bayesiana

In [77]:
search_spaces = {
    'svc__C': Real(1e-4, 1e+4, prior='log-uniform') 
}
cv_strategy = KFold(n_splits=10, shuffle=True, random_state=42)

opt1 = BayesSearchCV(
    estimator=pipeline,
    search_spaces=search_spaces,
    n_iter=10,  # 50 iteraciones de búsqueda
    scoring='roc_auc',
    cv=cv_strategy,
    n_jobs=-1,
    random_state=42,
    verbose=1 # Muestra el progreso
)
opt1.fit(X, y)

Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits


BayesSearchCV(cv=KFold(n_splits=10, random_state=42, shuffle=True),
              estimator=Pipeline(steps=[('preprocessor',
                                         ColumnTransformer(transformers=[('num',
                                                                          StandardScaler(),
                                                                          ['Patient_ID',
                                                                           'Age',
                                                                           'Tumor_Size',
                                                                           'Survival_Rate',
                                                                           'Tumor_Growth_Rate'])])),
                                        ('svc',
                                         SVC(C=0.19091131576909443,
                                             kernel='linear',
                                             probability=True))]),
              n_iter=10, n_jobs=-1, random_state=42, scoring='roc_auc',
              search_spaces={'svc__C': Real(low=0.0001, high=10000.0, prior='log-uniform', transform='normalize')},
              verbose=1)

In [94]:
print(f"El mejor score (ROC AUC) es: {opt1.best_score_}")
print(f"Los mejores hiperparámetros son: {opt1.best_params_}")

El mejor score (ROC AUC) es: 0.48905989039587244
Los mejores hiperparámetros son: OrderedDict({'svc__C': 0.19091131576909443})


El score bajó un poco en comparación con la primera optimización, por lo que ahora intentaremos con el nuevo valor de C

In [95]:
svc=SVC(kernel='linear', probability=True, C=0.19091131576909443)
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('svc', SVC(kernel='linear', probability=True))
])
pipeline.fit(X,y)
pipeline.score(X,y)
print(pipeline.score(X,y))

kfolds=10
scores=cross_val_score(pipeline,X,y,
                       scoring='roc_auc_ovr',
                       cv=kfolds)

0.537


In [96]:
mean_score=np.mean(scores)
mean_score

0.5126253360336034

In [97]:
std_score=np.std(scores)
std_score

0.02160459892006152

In [98]:
search_spaces = {
    'svc__C': Real(1e-4, 1e+4, prior='log-uniform') 
}
cv_strategy = KFold(n_splits=10, shuffle=True, random_state=42)

opt2 = BayesSearchCV(
    estimator=pipeline,
    search_spaces=search_spaces,
    n_iter=10,  # 50 iteraciones de búsqueda
    scoring='roc_auc',
    cv=cv_strategy,
    n_jobs=-1,
    random_state=42,
    verbose=1 # Muestra el progreso
)
opt2.fit(X, y)

Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits


BayesSearchCV(cv=KFold(n_splits=10, random_state=42, shuffle=True),
              estimator=Pipeline(steps=[('preprocessor',
                                         ColumnTransformer(transformers=[('num',
                                                                          StandardScaler(),
                                                                          ['Patient_ID',
                                                                           'Age',
                                                                           'Tumor_Size',
                                                                           'Survival_Rate',
                                                                           'Tumor_Growth_Rate']),
                                                                         ('cat',
                                                                          OneHotEncoder(handle_unknown='ignore'),
                                                                          ['Gender',
                                                                           'Location',
                                                                           'Histology',
                                                                           'Stage',
                                                                           'Symptom_1',
                                                                           'Symptom_2',
                                                                           'Symptom_3',
                                                                           'Radiation_Treatment',
                                                                           'Surgery_Performed',
                                                                           'Chemotherapy',
                                                                           'Family_History',
                                                                           'MRI_Result',
                                                                           'Follow_Up_Required'])])),
                                        ('svc',
                                         SVC(kernel='linear',
                                             probability=True))]),
              n_iter=10, n_jobs=-1, random_state=42, scoring='roc_auc',
              search_spaces={'svc__C': Real(low=0.0001, high=10000.0, prior='log-uniform', transform='normalize')},
              verbose=1)

In [99]:
print(f"El mejor score (ROC AUC) es: {opt2.best_score_}")
print(f"Los mejores hiperparámetros son: {opt2.best_params_}")

El mejor score (ROC AUC) es: 0.49471370138433296
Los mejores hiperparámetros son: OrderedDict({'svc__C': 4403.762515730915})
